# 리뷰 분석

## 리뷰 키워드 추출

https://brand.naver.com/bang-olufsen/products/5494514262?NaPm=ct%3Dm9xrwf0e%7Cci%3DER536a2266%2D225e%2D11f0%2Dbb8c%2D86176afd1675%7Ctr%3Dpla%7Chk%3Dc08021a432bb4ba306e7b080229e641d17140eb9%7Cnacn%3Dk9QOBkQAcghs#REVIEW

https://andar.co.kr/product/detail.html?product_no=12168&cate_no=2017&display_group=1

아래 고객 리뷰를 최대 5개 정도로 요약해서 keyword로 넣어줘
keyword는 항상 주어 + 서술어 형태로 ~해요체로 부탁해
부사는 넣지말아줘
상품에 대한 내용만 keyword로 넣어줘
욕설이 있으면 hasBadWords를 true로 추출해줘
강성 클레임 내용이 있으면 hasSevereClaim를 true로 추출해줘
주요 키워드와 원본 텍스트를 매칭하여 origianlText에 넣어줘
origianlText에 부정적인 내용이 있으면 positive를 false야
JSON 형태로 응답해줘. 다른 코멘트는 하지마

[
{
"keyword": "",
"originalText": "",
"positive": true,
"hasBadWords": false,
"hasSevereClaim: false,
},
]

- 고객 리뷰 : 먼저 구매하기 이전에 청음샵에서 여러 제품들과 비교해보면서 디자인은 당근빠따고 착용감이랑 소리 부분에서 부드럽다고 느껴 구매한 제 첫 헤드폰이라 만족스럽습니다. . 그냥 주관적으로 봤을때 전체적으로 소리 밸런스가 잘 잡혀있다고 느꼈고 여러 악기가 잘 들렸습니다. 특시 재즈나 클래식 장르에서 편안함이 느껴지네요. 저음이 살짝 약해 베이스같은 거 들을때 만족 안될 수 있어요. 다만 앱으로 조절하면 그나마 나아집니다. 편안한 무드로 중음~고음 음악 감상할때 추천

## 리뷰

먼저 구매하기 이전에 청음샵에서 여러 제품들과 비교해보면서 디자인은 당근빠따고 착용감이랑 소리 부분에서 부드럽다고 느껴 구매한 제 첫 헤드폰이라 만족스럽습니다. . 그냥 주관적으로 봤을때 전체적으로 소리 밸런스가 잘 잡혀있다고 느꼈고 여러 악기가 잘 들렸습니다. 특시 재즈나 클래식 장르에서 편안함이 느껴지네요. 저음이 살짝 약해 베이스같은 거 들을때 만족 안될 수 있어요. 다만 앱으로 조절하면 그나마 나아집니다. 편안한 무드로 중음~고음 음악 감상할때 추천

매번 캠핑 다니며 제주에서만 주문해 먹다가 여기 우연히 알게되었는데, 고기 정말 좋네요.
신선한 비계의 아삭함과 부드러운 살코기.
이런 고기는 다른 소스, 반찬 필요없이
그냥 소금에만 살짝 찍어 먹어도 훌륭해요.
잘 먹었습니다 ^^


In [1]:
import os

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

from poc.utils import stream_response

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY", "")

review_contents = """
먼저 구매하기 이전에 청음샵에서 여러 제품들과 비교해보면서 디자인은 당근빠따고 착용감이랑 소리 부분에서 부드럽다고 느껴 구매한 제 첫 헤드폰이라 만족스럽습니다. . 그냥 주관적으로 봤을때 전체적으로 소리 밸런스가 잘 잡혀있다고 느꼈고 여러 악기가 잘 들렸습니다. 특시 재즈나 클래식 장르에서 편안함이 느껴지네요. 저음이 살짝 약해 베이스같은 거 들을때 만족 안될 수 있어요. 다만 앱으로 조절하면 그나마 나아집니다. 편안한 무드로 중음~고음 음악 감상할때 추천
배송 받았는데 천하의 몹쓸놈들!!!!! 내 돈내놔라 이놈들아!! 이런 쓰레기를 팔면 지옥에 떨어질것이야!!

"""

chat_template = ChatPromptTemplate.from_messages(
    [
        "user",
        """
아래 고객 리뷰를 최대 5개 정도로 요약해서 keyword로 넣어줘
keyword는 항상 주어 + 서술어 형태로 ~해요체로 부탁해
부사는 넣지말아줘
상품에 대한 내용만 keyword로 넣어줘
욕설이 있으면 hasBadWords를 true로 추출해줘
강성 클레임 내용이 있으면 hasSevereClaim를 true로 추출해줘
주요 키워드와 원본 텍스트를 매칭하여 origianlText에 넣어줘
origianlText에 부정적인 내용이 있으면 positive를 false야
JSON 형태로 응답해줘. 다른 코멘트는 하지마

[
    {{
        "keyword": "",
        "originalText": "",
        "positive": true,
        "hasBadWords": false,
        "hasSevereClaim: false,
    }},
]
        
        {input}""",
    ]
)

messages = chat_template.format_messages(input=review_contents)

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

chain = chat_template | llm | StrOutputParser()

model_stream_response = chain.stream(messages)

response = stream_response(model_stream_response, return_output=True)

response


```json
[
    {
        "keyword": "디자인이 예뻐요",
        "originalText": "디자인은 당근빠따고",
        "positive": true,
        "hasBadWords": false,
        "hasSevereClaim": false
    },
    {
        "keyword": "착용감이 부드러워요",
        "originalText": "착용감이랑 소리 부분에서 부드럽다고 느껴 구매한 제 첫 헤드폰이라 만족스럽습니다.",
        "positive": true,
        "hasBadWords": false,
        "hasSevereClaim": false
    },
    {
        "keyword": "소리밸런스가 잡혀있어요",
        "originalText": "전체적으로 소리 밸런스가 잘 잡혀있다고 느꼈고 여러 악기가 잘 들렸습니다.",
        "positive": true,
        "hasBadWords": false,
        "hasSevereClaim": false
    },
    {
        "keyword": "재즈나 클래식에 편안해요",
        "originalText": "특시 재즈나 클래식 장르에서 편안함이 느껴지네요.",
        "positive": true,        "hasBadWords": false,
        "hasSevereClaim": false
    },
    {
        "keyword": "저음이 약해요",
        "originalText": "저음이 살짝 약해 베이스같은 거 들을때 만족 안될 수 있어요.",
        "positive": false,
        "hasBadWords": false,
        "hasSevereClaim": false
    },
    {
        "keyword"

'```json\n[\n    {\n        "keyword": "디자인이 예뻐요",\n        "originalText": "디자인은 당근빠따고",\n        "positive": true,\n        "hasBadWords": false,\n        "hasSevereClaim": false\n    },\n    {\n        "keyword": "착용감이 부드러워요",\n        "originalText": "착용감이랑 소리 부분에서 부드럽다고 느껴 구매한 제 첫 헤드폰이라 만족스럽습니다.",\n        "positive": true,\n        "hasBadWords": false,\n        "hasSevereClaim": false\n    },\n    {\n        "keyword": "소리밸런스가 잡혀있어요",\n        "originalText": "전체적으로 소리 밸런스가 잘 잡혀있다고 느꼈고 여러 악기가 잘 들렸습니다.",\n        "positive": true,\n        "hasBadWords": false,\n        "hasSevereClaim": false\n    },\n    {\n        "keyword": "재즈나 클래식에 편안해요",\n        "originalText": "특시 재즈나 클래식 장르에서 편안함이 느껴지네요.",\n        "positive": true,        "hasBadWords": false,\n        "hasSevereClaim": false\n    },\n    {\n        "keyword": "저음이 약해요",\n        "originalText": "저음이 살짝 약해 베이스같은 거 들을때 만족 안될 수 있어요.",\n        "positive": false,\n        "hasBadWords": false,\n        "hasSevereClaim": f

In [2]:
from pprint import pprint

import requests

# https://thirtymall.com/detail?id=124118780
url: str = "https://shop-api.e-ncp.com/products/124118780/product-reviews"
params: dict[str, str | int] = {
    "order.by": "REGISTER_YMDT",
    "order.direction": "DESC",
    "pageNumber": 1,
    "pageSize": 3,
    "hasTotalCount": "true",
}
headers: dict[str, str] = {
    "accept": "application/json, text/plain, */*",
    "Version": "1.0",
    "clientid": "UoI6WXPCVmuu7u/mv6tH2g==",
}

response = requests.get(url, params=params, headers=headers)
data = response.json()
pprint(data)


{'items': [{'bestReviewYn': 'N',
            'blindReportCnt': 0,
            'brandName': '참앤들황토농원',
            'brandNameEn': '참앤들황토농원',
            'cancelReportable': 'N',
            'commentCount': 0,
            'content': '저렴하게 잘 샀습니다. 처음 먹어 보지만 맛 괜찮네요.',
            'expelled': False,
            'externalReview': False,
            'extraJson': '',
            'fileUrls': [],
            'givenAccumulationYn': 'N',
            'imageUrl': '//thirtymall.cdn-nhncommerce.com/Mall-No-lKjO/20250123/155016.159672000/유기농레몬즙x10박스.jpg',
            'imageUrlType': 'IMAGE_URL',
            'isDeletedProductReview': False,
            'memberGradeDisplayInfo': None,
            'memberGroupDisplayInfo': None,
            'memberId': 'my****@naver.com',
            'memberName': '신*영',
            'myReview': False,
            'nickname': '',
            'orderedOption': {'addPrice': 0,
                              'inputs': [],
                              'optionName': '택1',
      

In [3]:
for review in data["items"]:
    content = review["content"]

    # 10자 미만이고 자음이나 숫자만 있는 경우 스킵
    if len(content) < 10 or all(
        char.isdigit() or char in "ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㄲㄸㅃㅆㅉ"
        for char in content
    ):
        continue

    messages = chat_template.format_messages(input=content)
    chain = chat_template | llm | StrOutputParser()
    model_stream_response = chain.stream(messages)
    response = stream_response(model_stream_response, return_output=True)

    # print(response)


```json
[
    {
        "keyword": "가격이 저렴해요",        "originalText": "저렴하게 잘 샀습니다.",
        "positive": true,
        "hasBadWords": false,
        "hasSevereClaim": false
    },
    {
        "keyword": "맛이 괜찮아요",
        "originalText": "처음 먹어 보지만 맛 괜찮네요.",
        "positive": true,        "hasBadWords": false,
        "hasSevereClaim": false
    }
]
``````json
[
    {
        "keyword": "가격이 저렴해요",
        "originalText": "저렴한가격에",
        "positive": true,
        "hasBadWords": false,
        "hasSevereClaim": false
    },
    {
        "keyword": "품질이 좋아요",
        "originalText": "좋은 품질의 제품을 살수있었어요",
        "positive": true,
        "hasBadWords": false,
        "hasSevereClaim": false
    }
]
``````json
[
    {
        "keyword": "필수품으로 구매해요",
        "originalText": "필수품으로 구매합니다",
        "positive": true,
        "hasBadWords": false,
        "hasSevereClaim": false
    }
]
```